# Taller: Análisis de Sentimientos en Tweets en Español (TASS 2018)

En este notebook vamos a construir paso a paso un clasificador de sentimiento para tweets en español utilizando el corpus de **TASS 2018**.  
El objetivo pedagógico es entender cada etapa típica de un *pipeline* de NLP aplicado a clasificación de texto:

1. Contexto del problema y del corpus TASS.
2. Configuración del entorno de trabajo.
3. Carga y exploración de los datos.
4. Preprocesamiento de texto en español.
5. Representación mediante **Bolsa de Palabras (Bag of Words)**.
6. Manejo del desbalance de clases.
7. Definición y entrenamiento de un modelo **Softmax (Regresión Logística Multinomial)**.
8. Evaluación con métricas de clasificación y matriz de confusión.

Cada bloque de código irá precedido de una breve explicación en español para que el flujo completo sea fácil de seguir en el taller.


## 1. Configuración del entorno

En este bloque definimos la ruta donde se encuentran los datos de TASS y añadimos el directorio actual al `sys.path` para poder importar módulos locales (por ejemplo, la clase `TextProcessing` que usaremos para el preprocesamiento).


In [3]:
import os
import sys
PATH = os.getcwd()
DIR_DATA = PATH + '{0}data{0}tass{0}'.format(os.sep)
sys.path.append(PATH) if PATH not in list(sys.path) else None
DIR_DATA

'C:\\Users\\epuerta\\OneDrive - Universidad Tecnológica de Bolívar\\Apps\\courseNLP\\examples\\data\\tass\\'

## 2. Contexto: TASS 2018

**TASS** es una campaña de evaluación y taller científico centrado en el **análisis de sentimientos en Twitter en español**.  
En la edición 2018, una de las tareas principales consiste en predecir la **polaridad global** de cada tweet (por ejemplo, positiva, negativa, neutra, etc.) a partir de texto corto, ruidoso y con variantes dialectales.

En este taller utilizaremos una versión de ese corpus para entrenar un modelo supervisado de clasificación de sentimiento.

Más información: <http://tass.sepln.org/2018/>


## 3. Importación de librerías

En el siguiente bloque importamos las librerías necesarias para:

- Manipular datos (`pandas`, `numpy`).
- Visualizar resultados (`matplotlib`, `seaborn`).
- Preprocesar y transformar texto (`TextProcessing`, `CountVectorizer`).
- Construir y evaluar modelos (`LogisticRegression`, métricas de `sklearn`).
- Tratar el desbalance de clases (`RandomOverSampler`).

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from logic.text_processing import TextProcessing
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, precision_score

## 4. Inicialización de utilidades de preprocesamiento

Instanciamos:

- `TextProcessing()`: encapsula las transformaciones de texto (limpieza, normalización, etc.).
- `LabelEncoder()`: utilidad para mapear etiquetas de texto a códigos numéricos si fuera necesario.

Esto nos permite mantener el preprocesamiento separado de la lógica del modelo.


In [5]:
tp = TextProcessing()
le = LabelEncoder()

Language: Text Processing
es: ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


## 5. Carga de los conjuntos de entrenamiento y prueba

A continuación cargamos:

- **`tass2018_es_train.csv`**: tweets etiquetados con su polaridad (conjunto de entrenamiento).
- **`tass2018_es_test.csv`**: tweets separados para evaluación final.

Es importante que durante el taller se explique el formato de las columnas (por ejemplo: identificador, contenido del tweet, etiqueta de polaridad).

In [6]:
data_train = pd.read_csv(DIR_DATA + 'tass2018_es_train.csv', sep=',')
data_train[:5]

,tweetid,user,content,date,lang,sentiment/polarity/value
0,768213876278165504,OnceBukowski,-Me caes muy bien \r\n-Tienes que jugar más pa...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\r\n\r\...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N


In [7]:
data_test = pd.read_csv(DIR_DATA + 'tass2018_es_test.csv', sep=',')
data_test[:5]

,tweetid,user,content,date,lang,sentiment/polarity/value
0,770976639173951488,noseashetero,@noseashetero 1000/10 de verdad a ti que voy a...,2016-08-31 13:28:49,es,P
1,771092421866389508,Templelx,@piscolabisaereo @HistoriaNG @SPosteguillo las...,2016-08-31 21:08:54,es,P
2,771092111429083136,esskuu94,"Al final han sido 3h Bueno, mañana tengo fies...",2016-08-31 21:07:40,es,P
3,771092070572449796,__ariadna9,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,2016-08-31 21:07:30,es,N
4,771094192508600320,_cristtina15_,@_MissChaotic_ ves ese brillo? es un coso que ...,2016-08-31 21:15:56,es,N


## 6. Preprocesamiento de texto

En esta etapa:

1. Tomamos el texto bruto de cada tweet (`content`).
2. Aplicamos `tp.transformer(...)` para:
   - Normalizar el texto (minúsculas, etc.).
   - Eliminar ruido típico de Twitter (URLs, menciones, signos repetidos, etc.).
   - Opcionalmente, manejar tildes, emojis o risas.

Guardamos:
- `x_train`, `x_test`: listas de tweets ya preprocesados.
- `y_train`, `y_test`: etiquetas de polaridad correspondientes.


In [8]:
x_train = [tp.transformer(row) for row in data_train['content'].tolist()]
#y_train = le.fit_transform(data_train['sentiment/polarity/value'])
y_train = data_train['sentiment/polarity/value']
len(x_train), len(y_train)

(1008, 1008)

In [9]:
# x_train

In [10]:
x_test = [tp.transformer(row) for row in data_test['content'].tolist()]
#y_test = le.fit_transform(data_test['sentiment/polarity/value'])
y_test = data_test['sentiment/polarity/value']
len(x_test), len(y_test)

(506, 506)

## 7. Representación: Bolsa de Palabras (Bag of Words)

El modelo no puede trabajar directamente con texto, así que lo convertimos en vectores numéricos.

Usamos `CountVectorizer` con:

- `analyzer='word'`
- `ngram_range=(1, 3)` para incluir unigramas, bigramas y trigramas.

Cada tweet se representa como un vector donde cada posición corresponde a una palabra o n-grama del vocabulario y el valor es su frecuencia en el tweet.


In [11]:
bow = CountVectorizer(analyzer='word', ngram_range=(1, 3))

In [12]:
x_train = bow.fit_transform(x_train)

In [13]:
x_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1008, 25996))

In [14]:
x_test = bow.transform(x_test)

In [15]:
x_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(506, 25996))

## 8. Análisis de distribución de clases

Antes de entrenar, observamos cuántos ejemplos hay por clase en entrenamiento y prueba.  
Esto nos permite detectar si el conjunto está **desbalanceado** (por ejemplo, muchas más opiniones neutrales que negativas), algo habitual en tareas reales.


In [16]:
print('**Sample train:', sorted(Counter(y_train).items()))

**Sample train: [('N', 418), ('NEU', 133), ('NONE', 139), ('P', 318)]


In [17]:
print('**Sample test:', sorted(Counter(y_test).items()))

**Sample test: [('N', 219), ('NEU', 69), ('NONE', 62), ('P', 156)]


## 9. Esquema de validación: ShuffleSplit (Validación Cruzada)

Para estimar el rendimiento del modelo de forma más robusta usamos `ShuffleSplit`:

- Se generan varias particiones aleatorias (aquí, 10).
- En cada partición se entrena con una parte de los datos y se evalúa con el resto.

Esto ayuda a reducir la dependencia de una única partición entrenamiento/prueba.


In [18]:
k_fold = ShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

## 10. Manejo del desbalance de clases: `RandomOverSampler`

Si algunas clases tienen muy pocos ejemplos, el modelo tiende a ignorarlas.
Con `RandomOverSampler`:

- Replicamos ejemplos de las clases minoritarias hasta equilibrar el conjunto.
- En este notebook se aplica tanto sobre `x_train` como sobre `x_test` para simplificar el ejercicio.

**Nota didáctica:** En un escenario real, el sobremuestreo se aplica únicamente sobre los datos de entrenamiento.


In [19]:
ros_train = RandomOverSampler(random_state=1000)
x_train, y_train = ros_train.fit_resample(x_train, y_train)

In [20]:
print('**OverSample train:', sorted(Counter(y_train).items()))

**OverSample train: [('N', 418), ('NEU', 418), ('NONE', 418), ('P', 418)]


In [21]:
ros_test = RandomOverSampler(random_state=1000)
x_test, y_test = ros_test.fit_resample(x_test, y_test)

In [22]:
print('**OverSample test:', sorted(Counter(y_test).items()))

**OverSample test: [('N', 219), ('NEU', 219), ('NONE', 219), ('P', 219)]


## 11. Modelo de clasificación: Regresión Logística Multinomial (Softmax)

Utilizamos `LogisticRegression` con:

- `multi_class="multinomial"` y `solver="lbfgs"`

Esto implementa un clasificador **Softmax** que aprende una probabilidad para cada clase de polaridad a partir del vector BOW del tweet.


In [23]:
softmax = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)

## 12. Métricas de evaluación

Vamos a registrar en cada iteración de validación cruzada:

- **Accuracy**: proporción de aciertos.
- **Recall (macro)**: capacidad de recuperar correctamente cada clase.
- **Precision (weighted)**: precisión ponderada por soporte de cada clase.
- **F1-score (weighted)**: equilibrio entre precisión y recall.

Esto nos da una visión más completa del comportamiento del modelo.


In [24]:
accuracies_scores = []
recalls_scores = []
precisions_scores = []
f1_scores = []

## 13. Entrenamiento y validación cruzada

En el siguiente bloque:

1. Generamos las particiones con `ShuffleSplit`.
2. Entrenamos el modelo `softmax` en los datos de entrenamiento de cada partición.
3. Predecimos sobre la parte de validación.
4. Calculamos las métricas y las guardamos para luego promediarlas.


In [25]:
for train_index, test_index in k_fold.split(x_train, y_train):
    data_train = x_train[train_index]
    target_train = y_train[train_index]
    
    data_test = x_train[test_index]
    target_test = y_train[test_index]

    softmax.fit(data_train, target_train)
    predict = softmax.predict(data_test)
    # Accuracy
    accuracy = accuracy_score(target_test, predict)
    accuracies_scores.append(accuracy)
    # Recall
    recall = recall_score(target_test, predict, average='macro')
    recalls_scores.append(recall)
    # Precision
    precision = precision_score(target_test, predict, average='weighted')
    precisions_scores.append(precision)
    # F1
    f1 = f1_score(target_test, predict, average='weighted')
    f1_scores.append(f1)

C:\Users\epuerta\AppData\Local\anaconda3\envs\python-nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\epuerta\AppData\Local\anaconda3\envs\python-nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\epuerta\AppData\Local\anaconda3\envs\python-nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\epuerta\AppData\Local\anaconda3

## 14. Resultados promedio en validación

Calculamos el valor promedio de cada métrica a lo largo de las particiones.


In [26]:
average_recall = round(np.mean(recalls_scores) * 100, 2)
average_precision = round(np.mean(precisions_scores) * 100, 2)
average_f1 = round(np.mean(f1_scores) * 100, 2)
average_accuracy = round(np.mean(accuracies_scores) * 100, 2)

In [27]:
average_recall

np.float64(80.24)

## 15. Evaluación final sobre el conjunto de prueba

Entrenamos el modelo final y evaluamos sobre el conjunto de prueba para obtener:

- **Reporte de clasificación** por clase.
- **Matriz de confusión**, que muestra cómo se confunden las clases entre sí.

Estas salidas permiten discutir en el taller:
- Qué clases se predicen mejor o peor.
- Cómo influye el desbalance y la representación BOW.
- Posibles mejoras (embeddings, modelos neuronales, ajuste de parámetros, etc.).


In [28]:
y_predict = []
for features in x_test:
    features = features.reshape(1, -1)
    value = softmax.predict(features)[0]
    y_predict.append(value)

classification = classification_report(y_test, y_predict)
confusion = confusion_matrix(y_predict, y_test)

In [29]:
output_result = {'F1-score': average_f1, 'Accuracy': average_accuracy, 'Recall': average_recall, 
                 'Precision': average_precision, 'Classification Report\n': classification, 
                 'Confusion Matrix\n': confusion}

In [30]:
for item, val in output_result.items():
    print('{0} {1}'.format(item, val))

F1-score 80.21
Accuracy 80.26
Recall 80.24
Precision 80.34
Classification Report
               precision    recall  f1-score   support

           N       0.35      0.69      0.47       219
         NEU       0.46      0.19      0.27       219
        NONE       0.42      0.18      0.25       219
           P       0.39      0.47      0.43       219

    accuracy                           0.38       876
   macro avg       0.41      0.38      0.35       876
weighted avg       0.41      0.38      0.35       876

Confusion Matrix
 [[151  92 109  78]
 [ 15  41  21  12]
 [ 15  14  39  25]
 [ 38  72  50 104]]


## 16. Próximos pasos sugeridos para el taller

Algunas extensiones que se pueden proponer a los participantes:

- Probar distintas configuraciones de `CountVectorizer` (solo unigramas, límite de vocabulario, stopwords, etc.).
- Comparar la Regresión Logística con otros clasificadores (SVM, árboles, redes neuronales sencillas).
- Analizar ejemplos mal clasificados a partir de la matriz de confusión.
- Integrar representaciones basadas en *embeddings* para comparar con la bolsa de palabras.

Con estas actividades, el notebook sirve como guía completa de un pipeline clásico de NLP aplicado a TASS 2018.
